In [29]:
import os
!pwd

/home/mehrajofficial000/venv/ml_project


In [30]:
#os.chdir("../")

In [31]:
!pwd

/home/mehrajofficial000/venv/ml_project


In [32]:
#entity 
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_path: Path

In [33]:
#configureation manager
from Classifier.constants import *
from Classifier.utils.utilities import read_yaml, create_directories

class ConfigureationManager:
    def __init__(self, config_file_path = CONFIG_FILE_PATH , params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        
        create_directories([self.config.artifacts_root])
    
    def get_prepare_callbacks_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_checkpoint_dir = os.path.dirname(config.checkpoint_model_path)
        create_directories([
            Path(model_checkpoint_dir),
            Path(config.tensorboard_root_log_dir),
        ])
        prepare_callbacks_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_path=Path(config.checkpoint_model_path)
        )
        return prepare_callbacks_config


In [34]:
#Components
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf 
import time 
import logging

class PrepareCallbacks:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config
    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir, 
            f"tb_logs_at-{timestamp}"
        )
        logging.info(f"Tensorboard log dir: {tb_running_log_dir}")
        return tf.keras.callbacks.TensorBoard(
            log_dir=tb_running_log_dir,
            histogram_freq=1, #Problem might occur 
            write_graph=True,  
            write_images=True,
            update_freq="epoch",
        )
    
    @property
    def _create_checkpoint_callback(self):
        logging.info(f"Checkpoint dir: --->>>>> creating checkpoint dir ----->")
        return tf.keras.callbacks.ModelCheckpoint(
            filepath = self.config.checkpoint_model_path,
            monitor = "val_accuracy", #problems might occur 
            save_best_only = True,
        )
    
    def get_callbacks(self):
        logging.info("Preparing callbacks")
        return [
            self._create_tb_callbacks,
            self._create_checkpoint_callback
        ]
         
        

In [ ]:
#pipeline

try :
    config = ConfigureationManager()
    prepare_callbacks_config = config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallbacks(config=prepare_callbacks_config)
    callbacks = prepare_callbacks.get_callbacks()
    logging.info(f"Callbacks: {callbacks}")
except Exception as e:
    logging.error(f"Error in loading configuration: {e}")
    raise e

[2025-08-02 11:04:38,279] INFO: utilities: YAML file config/config.yaml loaded successfully.]
[2025-08-02 11:04:38,282] INFO: utilities: YAML file params.yaml loaded successfully.]
[2025-08-02 11:04:38,285] INFO: utilities: Directory craeted at artifacts]
[2025-08-02 11:04:38,286] INFO: utilities: Directory craeted at artifacts/prepare_callbacks/checkpoint_dir]
[2025-08-02 11:04:38,288] INFO: utilities: Directory craeted at artifacts/prepare_callbacks/tensorboard_log_dir]
[2025-08-02 11:04:38,290] INFO: 255290885: Preparing callbacks]
[2025-08-02 11:04:38,292] INFO: 255290885: Tensorboard log dir: artifacts/prepare_callbacks/tensorboard_log_dir/tb_logs_at-2025-08-02-11-04-38]


[2025-08-02 11:04:38,293] INFO: 255290885: Checkpoint dir: --->>>>> creating checkpoint dir ----->]
[2025-08-02 11:04:38,295] INFO: 660380686: Callbacks: [<keras.src.callbacks.tensorboard.TensorBoard object at 0x7f038c237590>, <keras.src.callbacks.model_checkpoint.ModelCheckpoint object at 0x7f038e860590>]]
